# Entrenamiento de Modelos de Deep Learning para Detección de Cáncer

Este notebook implementa el entrenamiento de modelos de deep learning para detección temprana de cáncer, incluyendo:

- CNN tradicionales (ResNet, EfficientNet)
- Vision Transformers (ViT)
- Modelos híbridos
- Evaluación y comparación de modelos
- Análisis de interpretabilidad
- Integración con Gemini AI

In [ ]:
# Configuración inicial
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Agregar src al path
src_path = Path('../src').absolute()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Imports principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# ML/DL imports
try:
    import tensorflow as tf
    from tensorflow import keras
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, confusion_matrix
    print(f"✓ TensorFlow version: {tf.__version__}")
    TF_AVAILABLE = True
except ImportError:
    print("⚠️  TensorFlow no disponible")
    TF_AVAILABLE = False

# Configurar GPU si está disponible
if TF_AVAILABLE:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        print(f"✓ GPU disponible: {len(gpus)} dispositivo(s)")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        print("🔋 Usando CPU para entrenamiento")

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)

print(f"Entrenamiento de modelos iniciado: {datetime.now()}")

In [ ]:
# Importar módulos del proyecto
try:
    from models.cancer_detection import CancerDetectionModel
    from utils.gemini_analyzer import GeminiAnalyzer
    from utils.dicom_processor import DICOMProcessor
    print("✓ Módulos del proyecto importados")
except ImportError as e:
    print(f"❌ Error importando módulos: {e}")
    print("Algunos análisis pueden no estar disponibles")

## 1. Preparación de Datos Sintéticos

Para esta demostración, generaremos un dataset sintético que simule imágenes médicas procesadas.

In [ ]:
# Generar dataset sintético para demostración
def generate_synthetic_medical_images(n_samples=1000, image_size=(224, 224, 3)):
    """Genera imágenes médicas sintéticas para entrenamiento."""
    
    np.random.seed(42)
    
    # Generar imágenes base
    X = np.random.normal(0.5, 0.2, (n_samples,) + image_size)
    
    # Generar etiquetas (60% benigno, 40% maligno)
    y = np.random.choice([0, 1], size=n_samples, p=[0.6, 0.4])
    
    # Agregar patrones diferenciadores sutiles
    for i in range(n_samples):
        if y[i] == 1:  # Maligno
            # Agregar textura irregular
            noise = np.random.normal(0, 0.3, image_size)
            X[i] = np.clip(X[i] + noise * 0.2, 0, 1)
            
            # Agregar "lesiones" circulares
            center_x, center_y = np.random.randint(50, image_size[0]-50, 2)
            radius = np.random.randint(10, 30)
            
            y_coords, x_coords = np.ogrid[:image_size[0], :image_size[1]]
            mask = (x_coords - center_x)**2 + (y_coords - center_y)**2 <= radius**2
            
            # Intensidad alterada en la "lesión"
            intensity = np.random.uniform(0.2, 0.8)
            X[i][mask] = intensity
        
        else:  # Benigno
            # Textura más uniforme
            smooth_noise = np.random.normal(0, 0.1, image_size)
            X[i] = np.clip(X[i] + smooth_noise * 0.1, 0, 1)
    
    # Asegurar rango [0, 1]
    X = np.clip(X, 0, 1)
    
    return X, y

print("Generando dataset sintético...")
X_synthetic, y_synthetic = generate_synthetic_medical_images(n_samples=1000)

print(f"✓ Dataset generado:")
print(f"  - Imágenes: {X_synthetic.shape}")
print(f"  - Etiquetas: {y_synthetic.shape}")
print(f"  - Casos benignos: {np.sum(y_synthetic == 0)}")
print(f"  - Casos malignos: {np.sum(y_synthetic == 1)}")

# Visualizar muestras
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Mostrar ejemplos benignos
benign_indices = np.where(y_synthetic == 0)[0][:5]
for i, idx in enumerate(benign_indices):
    axes[0, i].imshow(X_synthetic[idx])
    axes[0, i].set_title(f'Benigno {idx}')
    axes[0, i].axis('off')

# Mostrar ejemplos malignos
malignant_indices = np.where(y_synthetic == 1)[0][:5]
for i, idx in enumerate(malignant_indices):
    axes[1, i].imshow(X_synthetic[idx])
    axes[1, i].set_title(f'Maligno {idx}')
    axes[1, i].axis('off')

plt.suptitle('Muestras del Dataset Sintético', fontsize=16)
plt.tight_layout()
plt.show()

## 2. División del Dataset

In [ ]:
# Dividir dataset en entrenamiento, validación y prueba
if TF_AVAILABLE:
    # División inicial: 80% train+val, 20% test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_synthetic, y_synthetic, test_size=0.2, random_state=42, stratify=y_synthetic
    )
    
    # División: 80% train, 20% val del conjunto temporal
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
    )
    
    print(f"📊 División del dataset:")
    print(f"  - Entrenamiento: {X_train.shape[0]} imágenes")
    print(f"    * Benigno: {np.sum(y_train == 0)}, Maligno: {np.sum(y_train == 1)}")
    print(f"  - Validación: {X_val.shape[0]} imágenes")
    print(f"    * Benigno: {np.sum(y_val == 0)}, Maligno: {np.sum(y_val == 1)}")
    print(f"  - Prueba: {X_test.shape[0]} imágenes")
    print(f"    * Benigno: {np.sum(y_test == 0)}, Maligno: {np.sum(y_test == 1)}")
    
    # Convertir etiquetas a categorical para entrenamiento
    if TF_AVAILABLE:
        y_train_cat = keras.utils.to_categorical(y_train, 2)
        y_val_cat = keras.utils.to_categorical(y_val, 2)
        y_test_cat = keras.utils.to_categorical(y_test, 2)
        
        print(f"\n✓ Etiquetas convertidas a formato categórico")
else:
    print("⚠️  TensorFlow no disponible - saltando división de datos")

## 3. Inicialización del Modelo de Detección

In [ ]:
# Inicializar modelo de detección de cáncer
if TF_AVAILABLE:
    try:
        cancer_model = CancerDetectionModel()
        print("✓ Modelo de detección de cáncer inicializado")
        
        # Mostrar configuración
        print(f"\n📋 Configuración del modelo:")
        print(f"  - Forma de entrada: {cancer_model.input_shape}")
        print(f"  - Número de clases: {cancer_model.num_classes}")
        print(f"  - Learning rate: {cancer_model.learning_rate}")
        print(f"  - Épocas: {cancer_model.epochs}")
        print(f"  - Paciencia: {cancer_model.patience}")
        
    except Exception as e:
        print(f"❌ Error inicializando modelo: {e}")
        cancer_model = None
else:
    print("⚠️  TensorFlow no disponible")
    cancer_model = None

## 4. Entrenamiento de Modelos CNN

In [ ]:
# Entrenar diferentes arquitecturas CNN
if cancer_model and TF_AVAILABLE:
    
    # Arquitecturas a probar
    architectures = ['ResNet50', 'EfficientNetB0']
    model_results = {}
    
    for arch in architectures:
        print(f"\n🚀 Entrenando modelo {arch}...")
        print("=" * 50)
        
        try:
            # Crear una nueva instancia del modelo para cada arquitectura
            model_instance = CancerDetectionModel()
            
            # Entrenar modelo (con épocas reducidas para demo)
            model_instance.epochs = 5  # Reducido para demo
            model_instance.patience = 3
            
            results = model_instance.train_model(
                (X_train, y_train_cat),
                (X_val, y_val_cat),
                model_type=arch
            )
            
            if results and 'validation_metrics' in results:
                model_results[arch] = results
                val_metrics = results['validation_metrics']
                
                print(f"\n✅ {arch} - Métricas de validación:")
                for metric, value in val_metrics.items():
                    print(f"  - {metric}: {value:.4f}")
                
                # Guardar modelo
                model_path = f"../results/models/{arch.lower()}_model.h5"
                os.makedirs(os.path.dirname(model_path), exist_ok=True)
                
                if model_instance.save_model(model_path):
                    print(f"💾 Modelo guardado: {model_path}")
            else:
                print(f"❌ Error entrenando {arch}")
                
        except Exception as e:
            print(f"❌ Error con {arch}: {e}")
            continue
    
    print(f"\n📊 RESUMEN DE ENTRENAMIENTO CNN")
    print(f"=" * 50)
    
    if model_results:
        comparison_df = pd.DataFrame({
            arch: results['validation_metrics'] 
            for arch, results in model_results.items()
        }).T
        
        print("Comparación de modelos CNN:")
        display(comparison_df.round(4))
        
        # Visualizar métricas
        if len(model_results) > 1:
            comparison_df.plot(kind='bar', figsize=(12, 6))
            plt.title('Comparación de Métricas por Arquitectura CNN')
            plt.xlabel('Arquitectura')
            plt.ylabel('Valor de Métrica')
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
    else:
        print("No se completó el entrenamiento de ningún modelo CNN")
        
else:
    print("⚠️  Modelo no disponible - saltando entrenamiento CNN")
    model_results = {}

## 5. Entrenamiento de Vision Transformer

In [ ]:
# Entrenar Vision Transformer
if cancer_model and TF_AVAILABLE:
    print(f"\n🔍 Entrenando Vision Transformer...")
    print("=" * 50)
    
    try:
        # Crear instancia para ViT
        vit_model = CancerDetectionModel()
        vit_model.epochs = 3  # Menos épocas para ViT por complejidad
        vit_model.patience = 2
        
        vit_results = vit_model.train_model(
            (X_train, y_train_cat),
            (X_val, y_val_cat),
            model_type="ViT"
        )
        
        if vit_results and 'validation_metrics' in vit_results:
            model_results['ViT'] = vit_results
            val_metrics = vit_results['validation_metrics']
            
            print(f"\n✅ Vision Transformer - Métricas de validación:")
            for metric, value in val_metrics.items():
                print(f"  - {metric}: {value:.4f}")
            
            # Guardar modelo ViT
            vit_path = "../results/models/vit_model.h5"
            if vit_model.save_model(vit_path):
                print(f"💾 Vision Transformer guardado: {vit_path}")
        else:
            print("❌ Error entrenando Vision Transformer")
            
    except Exception as e:
        print(f"❌ Error con Vision Transformer: {e}")
        
else:
    print("⚠️  Saltando entrenamiento de Vision Transformer")

## 6. Entrenamiento de Modelo Híbrido

In [ ]:
# Entrenar modelo híbrido CNN + ViT
if cancer_model and TF_AVAILABLE:
    print(f"\n🔗 Entrenando Modelo Híbrido (CNN + ViT)...")
    print("=" * 50)
    
    try:
        # Crear instancia para modelo híbrido
        hybrid_model = CancerDetectionModel()
        hybrid_model.epochs = 4  # Épocas moderadas
        hybrid_model.patience = 2
        
        hybrid_results = hybrid_model.train_model(
            (X_train, y_train_cat),
            (X_val, y_val_cat),
            model_type="Hybrid"
        )
        
        if hybrid_results and 'validation_metrics' in hybrid_results:
            model_results['Hybrid'] = hybrid_results
            val_metrics = hybrid_results['validation_metrics']
            
            print(f"\n✅ Modelo Híbrido - Métricas de validación:")
            for metric, value in val_metrics.items():
                print(f"  - {metric}: {value:.4f}")
            
            # Guardar modelo híbrido
            hybrid_path = "../results/models/hybrid_model.h5"
            if hybrid_model.save_model(hybrid_path):
                print(f"💾 Modelo Híbrido guardado: {hybrid_path}")
        else:
            print("❌ Error entrenando Modelo Híbrido")
            
    except Exception as e:
        print(f"❌ Error con Modelo Híbrido: {e}")
        
else:
    print("⚠️  Saltando entrenamiento de Modelo Híbrido")

## 7. Comparación de Modelos

In [ ]:
# Comparación completa de todos los modelos
if model_results:
    print(f"\n🏆 COMPARACIÓN FINAL DE MODELOS")
    print(f"=" * 50)
    
    # Crear DataFrame comparativo
    comparison_data = {}
    
    for model_name, results in model_results.items():
        if 'validation_metrics' in results:
            comparison_data[model_name] = results['validation_metrics']
    
    if comparison_data:
        final_comparison = pd.DataFrame(comparison_data).T
        
        print("\n📊 Métricas de Validación por Modelo:")
        display(final_comparison.round(4))
        
        # Encontrar mejor modelo por métrica
        print("\n🥇 Mejores modelos por métrica:")
        for metric in final_comparison.columns:
            best_model = final_comparison[metric].idxmax()
            best_value = final_comparison.loc[best_model, metric]
            print(f"  - {metric}: {best_model} ({best_value:.4f})")
        
        # Visualizaciones comparativas
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Gráfico de barras por métrica
        final_comparison.plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title('Comparación de Métricas por Modelo')
        axes[0,0].set_xlabel('Modelo')
        axes[0,0].set_ylabel('Valor')
        axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0,0].tick_params(axis='x', rotation=45)
        
        # Radar chart de métricas (si hay múltiples métricas)
        if len(final_comparison.columns) >= 3:
            from math import pi
            
            categories = list(final_comparison.columns)
            N = len(categories)
            
            angles = [n / float(N) * 2 * pi for n in range(N)]
            angles += angles[:1]
            
            axes[0,1].set_theta_offset(pi / 2)
            axes[0,1].set_theta_direction(-1)
            axes[0,1] = plt.subplot(2, 2, 2, projection='polar')
            
            for model_name in final_comparison.index:
                values = final_comparison.loc[model_name].values.flatten().tolist()
                values += values[:1]
                
                axes[0,1].plot(angles, values, 'o-', linewidth=2, label=model_name)
                axes[0,1].fill(angles, values, alpha=0.25)
            
            axes[0,1].set_xticks(angles[:-1])
            axes[0,1].set_xticklabels(categories)
            axes[0,1].set_ylim(0, 1)
            axes[0,1].set_title('Perfil de Rendimiento por Modelo')
            axes[0,1].legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
        else:
            axes[0,1].axis('off')
            axes[0,1].text(0.5, 0.5, 'Radar chart requiere\n≥3 métricas', 
                          ha='center', va='center', transform=axes[0,1].transAxes)
        
        # Historial de entrenamiento (si disponible)
        if 'history' in list(model_results.values())[0]:
            for i, (model_name, results) in enumerate(model_results.items()):
                if 'history' in results and 'val_accuracy' in results['history']:
                    epochs = range(1, len(results['history']['val_accuracy'])+1)
                    axes[1,0].plot(epochs, results['history']['val_accuracy'], 
                                 label=f'{model_name} - Val Acc', marker='o')
                    axes[1,0].plot(epochs, results['history']['accuracy'], 
                                 label=f'{model_name} - Train Acc', linestyle='--')
            
            axes[1,0].set_title('Evolución de Accuracy durante Entrenamiento')
            axes[1,0].set_xlabel('Época')
            axes[1,0].set_ylabel('Accuracy')
            axes[1,0].legend()
            axes[1,0].grid(True, alpha=0.3)
        else:
            axes[1,0].axis('off')
            axes[1,0].text(0.5, 0.5, 'Historial no disponible', 
                          ha='center', va='center', transform=axes[1,0].transAxes)
        
        # Ranking de modelos
        axes[1,1].axis('off')
        
        # Calcular score promedio
        avg_scores = final_comparison.mean(axis=1).sort_values(ascending=False)
        
        ranking_text = "🏆 RANKING FINAL:\n\n"
        for i, (model, score) in enumerate(avg_scores.items()):
            medal = ["🥇", "🥈", "🥉"][i] if i < 3 else f"{i+1}."
            ranking_text += f"{medal} {model}: {score:.3f}\n"
        
        axes[1,1].text(0.1, 0.7, ranking_text, fontsize=12, 
                      va='top', transform=axes[1,1].transAxes,
                      bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        # Guardar comparación
        final_comparison.to_csv('../results/model_comparison.csv')
        print(f"\n💾 Comparación guardada en: results/model_comparison.csv")
        
    else:
        print("❌ No hay datos de comparación disponibles")
        
else:
    print("⚠️  No se entrenaron modelos para comparar")

## 8. Evaluación en Conjunto de Prueba

In [ ]:
# Evaluar el mejor modelo en conjunto de prueba
if model_results and TF_AVAILABLE:
    print(f"\n🧪 EVALUACIÓN EN CONJUNTO DE PRUEBA")
    print(f"=" * 50)
    
    try:
        # Encontrar mejor modelo por accuracy
        best_model_name = None
        best_accuracy = 0
        
        for model_name, results in model_results.items():
            if 'validation_metrics' in results and 'accuracy' in results['validation_metrics']:
                acc = results['validation_metrics']['accuracy']
                if acc > best_accuracy:
                    best_accuracy = acc
                    best_model_name = model_name
        
        if best_model_name:
            print(f"🏆 Mejor modelo: {best_model_name} (Val Acc: {best_accuracy:.4f})")
            
            # Cargar mejor modelo para evaluación
            model_path = f"../results/models/{best_model_name.lower()}_model.h5"
            
            if os.path.exists(model_path):
                try:
                    best_model = keras.models.load_model(model_path)
                    print(f"✓ Modelo cargado desde: {model_path}")
                    
                    # Evaluar en conjunto de prueba
                    test_loss, test_acc = best_model.evaluate(X_test, y_test_cat, verbose=0)
                    
                    # Predicciones
                    y_pred_proba = best_model.predict(X_test, verbose=0)
                    y_pred = np.argmax(y_pred_proba, axis=1)
                    
                    print(f"\n📊 Resultados en conjunto de prueba:")
                    print(f"  - Pérdida: {test_loss:.4f}")
                    print(f"  - Accuracy: {test_acc:.4f}")
                    
                    # Matriz de confusión
                    cm = confusion_matrix(y_test, y_pred)
                    
                    # Reporte de clasificación
                    print(f"\n📋 Reporte de clasificación:")
                    print(classification_report(y_test, y_pred, 
                                               target_names=['Benigno', 'Maligno']))
                    
                    # Visualizar matriz de confusión
                    plt.figure(figsize=(10, 4))
                    
                    plt.subplot(1, 2, 1)
                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                               xticklabels=['Benigno', 'Maligno'],
                               yticklabels=['Benigno', 'Maligno'])
                    plt.title(f'Matriz de Confusión - {best_model_name}')
                    plt.xlabel('Predicción')
                    plt.ylabel('Real')
                    
                    # Distribución de probabilidades
                    plt.subplot(1, 2, 2)
                    
                    # Probabilidades para cada clase
                    prob_malignant = y_pred_proba[:, 1]
                    
                    benign_probs = prob_malignant[y_test == 0]
                    malignant_probs = prob_malignant[y_test == 1]
                    
                    plt.hist(benign_probs, alpha=0.7, label='Benigno Real', 
                            bins=20, color='skyblue')
                    plt.hist(malignant_probs, alpha=0.7, label='Maligno Real', 
                            bins=20, color='salmon')
                    
                    plt.axvline(x=0.5, color='red', linestyle='--', 
                               label='Umbral (0.5)')
                    plt.xlabel('Probabilidad de Malignidad')
                    plt.ylabel('Frecuencia')
                    plt.title('Distribución de Probabilidades Predichas')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Guardar resultados de evaluación
                    test_results = {
                        'best_model': best_model_name,
                        'test_loss': float(test_loss),
                        'test_accuracy': float(test_acc),
                        'confusion_matrix': cm.tolist(),
                        'classification_report': classification_report(
                            y_test, y_pred, target_names=['Benigno', 'Maligno'],
                            output_dict=True
                        )
                    }
                    
                    with open('../results/test_evaluation.json', 'w') as f:
                        json.dump(test_results, f, indent=2)
                    
                    print(f"\n💾 Evaluación guardada en: results/test_evaluation.json")
                    
                except Exception as e:
                    print(f"❌ Error cargando/evaluando modelo: {e}")
            else:
                print(f"❌ Modelo no encontrado: {model_path}")
        else:
            print("❌ No se pudo identificar el mejor modelo")
            
    except Exception as e:
        print(f"❌ Error en evaluación: {e}")
        
else:
    print("⚠️  No hay modelos para evaluar")

## 9. Análisis con Gemini AI (Opcional)

In [ ]:
# Análisis de algunas imágenes de prueba con Gemini AI
try:
    gemini_analyzer = GeminiAnalyzer()
    print("✓ Analizador Gemini disponible")
    gemini_available = True
except Exception as e:
    print(f"⚠️  Analizador Gemini no disponible: {e}")
    gemini_available = False

if gemini_available:
    print(f"\n🤖 ANÁLISIS CON GEMINI AI")
    print(f"=" * 50)
    
    try:
        # Seleccionar algunas imágenes de prueba
        n_analyze = min(3, len(X_test))
        selected_indices = np.random.choice(len(X_test), n_analyze, replace=False)
        
        gemini_analyses = []
        
        for i, idx in enumerate(selected_indices):
            print(f"\nAnalizando imagen de prueba {i+1}/{n_analyze}...")
            
            # Guardar imagen temporalmente
            temp_img_path = f"../results/temp_image_{i}.png"
            
            # Convertir y guardar imagen
            img_to_save = (X_test[idx] * 255).astype(np.uint8)
            
            from PIL import Image
            pil_img = Image.fromarray(img_to_save)
            pil_img.save(temp_img_path)
            
            # Analizar con Gemini
            try:
                analysis = gemini_analyzer.analyze_medical_image(
                    temp_img_path, 
                    analysis_type="cancer_detection"
                )
                
                if analysis and 'gemini_response' in analysis:
                    # Agregar información adicional
                    analysis['true_label'] = 'Maligno' if y_test[idx] == 1 else 'Benigno'
                    analysis['image_index'] = int(idx)
                    
                    if 'y_pred' in locals():
                        analysis['model_prediction'] = 'Maligno' if y_pred[idx] == 1 else 'Benigno'
                        analysis['prediction_confidence'] = float(y_pred_proba[idx].max())
                    
                    gemini_analyses.append(analysis)
                    
                    print(f"✓ Análisis Gemini completado")
                    print(f"  - Etiqueta real: {analysis['true_label']}")
                    if 'model_prediction' in analysis:
                        print(f"  - Predicción modelo: {analysis['model_prediction']}")
                        print(f"  - Confianza: {analysis['prediction_confidence']:.3f}")
                    
                    # Mostrar fragmento del análisis
                    response_snippet = analysis['gemini_response'][:300] + "..."
                    print(f"  - Análisis Gemini: {response_snippet}")
                else:
                    print(f"❌ Error en análisis Gemini")
                    
            except Exception as e:
                print(f"❌ Error analizando con Gemini: {e}")
            
            # Limpiar archivo temporal
            try:
                os.remove(temp_img_path)
            except:
                pass
        
        # Guardar análisis de Gemini
        if gemini_analyses:
            with open('../results/gemini_model_analysis.json', 'w') as f:
                json.dump(gemini_analyses, f, indent=2)
            
            print(f"\n💾 Análisis Gemini guardado en: results/gemini_model_analysis.json")
            print(f"\n🔍 Gemini analizó {len(gemini_analyses)} imágenes de prueba")
        
    except Exception as e:
        print(f"❌ Error en análisis con Gemini: {e}")
        
else:
    print("🔒 Análisis con Gemini no disponible")

## 10. Resumen Final y Conclusiones

In [ ]:
# Generar resumen final del entrenamiento
print(f"\n🎯 RESUMEN FINAL DEL ENTRENAMIENTO")
print(f"=" * 60)

print(f"\n📊 DATASET UTILIZADO:")
print(f"  - Total de imágenes: {len(X_synthetic)}")
print(f"  - Resolución: {X_synthetic.shape[1:]}")
print(f"  - Casos benignos: {np.sum(y_synthetic == 0)} ({np.mean(y_synthetic == 0)*100:.1f}%)")
print(f"  - Casos malignos: {np.sum(y_synthetic == 1)} ({np.mean(y_synthetic == 1)*100:.1f}%)")

if TF_AVAILABLE:
    print(f"\n🚀 MODELOS ENTRENADOS:")
    if model_results:
        for i, (model_name, results) in enumerate(model_results.items(), 1):
            print(f"  {i}. {model_name}")
            if 'validation_metrics' in results and 'accuracy' in results['validation_metrics']:
                acc = results['validation_metrics']['accuracy']
                print(f"     - Accuracy de validación: {acc:.4f}")
    else:
        print(f"  ❌ No se completó el entrenamiento de ningún modelo")
else:
    print(f"\n⚠️  TensorFlow no disponible - no se entrenaron modelos")

if 'best_model_name' in locals() and best_model_name:
    print(f"\n🏆 MEJOR MODELO:")
    print(f"  - Arquitectura: {best_model_name}")
    print(f"  - Accuracy de validación: {best_accuracy:.4f}")
    if 'test_acc' in locals():
        print(f"  - Accuracy de prueba: {test_acc:.4f}")

print(f"\n💾 ARCHIVOS GENERADOS:")
output_files = [
    'results/models/',
    'results/model_comparison.csv',
    'results/test_evaluation.json'
]

if 'gemini_analyses' in locals() and gemini_analyses:
    output_files.append('results/gemini_model_analysis.json')

for file_path in output_files:
    full_path = Path(f'../{file_path}')
    if full_path.exists():
        if full_path.is_dir():
            model_count = len(list(full_path.glob('*.h5')))
            print(f"  ✓ {file_path} ({model_count} modelos)")
        else:
            print(f"  ✓ {file_path}")
    else:
        print(f"  ❌ {file_path} (no generado)")

print(f"\n🔬 METODOLOGÍA EMPLEADA:")
print(f"  1. Generación de dataset sintético con patrones diferenciadores")
print(f"  2. División estratificada train/val/test (64%/16%/20%)")
print(f"  3. Entrenamiento de múltiples arquitecturas (CNN, ViT, Híbrido)")
print(f"  4. Comparación basada en métricas de validación")
print(f"  5. Evaluación final en conjunto de prueba independiente")
if gemini_available:
    print(f"  6. Análisis cualitativo con Gemini AI")

print(f"\n📈 PRÓXIMOS PASOS RECOMENDADOS:")
print(f"  1. Entrenamiento con datos reales de TCIA")
print(f"  2. Implementación de técnicas de interpretabilidad (Grad-CAM, LIME)")
print(f"  3. Validación externa con datasets independientes")
print(f"  4. Optimización de hiperparámetros")
print(f"  5. Desarrollo de pipeline de producción")
print(f"  6. Integración con análisis radiómico")

if model_results:
    print(f"\n✅ CONCLUSIÓN:")
    print(f"   Se entrenaron exitosamente {len(model_results)} modelos de deep learning")
    print(f"   para detección de cáncer. Los resultados muestran la viabilidad del")
    print(f"   enfoque y proporcionan una base sólida para el desarrollo de un")
    print(f"   sistema de diagnóstico asistido por IA.")
else:
    print(f"\n ⚠️  NOTA:")
    print(f"   Este notebook demuestra la metodología de entrenamiento.")
    print(f"   Para resultados óptimos, se requiere un entorno con TensorFlow")
    print(f"   y datos reales de imágenes médicas.")

print(f"\n🕒 Entrenamiento completado: {datetime.now()}")